In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


app = dash.Dash(__name__)

uniquelaunchsites = spacex_df['Launch Site'].unique().tolist()
lsites = []
lsites.append({'label': 'All Sites', 'value': 'ALL'})
for site in uniquelaunchsites:
 lsites.append({'label': site, 'value': site})                                 
                           


app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                
                                

                    
                                dcc.Dropdown(id='site_dropdown',options=lsites,placeholder='Select a Launch Site here', searchable = True , value = 'All Sites'),
                                html.Br(),

                       
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                             
                                dcc.RangeSlider(
                                    id='payload_slider',
                                    min=0,
                                    max=10000,
                                    step=1000,  
                                    marks = {
                                            0: '0 kg',
                                            1000: '1000 ',
                                            2000: '2000 ',
                                            3000: '3000 ',
                                            4000: '4000 ',
                                            5000: '5000 ',
                                            6000: '6000 ',  
                                            7000: '7000 ',
                                            8000: '8000 ',
                                            9000: '9000 ',
                                            10000: '10000 '                                         
                                    },

                                    value=[min_payload,max_payload]
                                ),
                            
                                html.Div(dcc.Graph(id='success-vs-payload-chart')),
                              
                                ])

@app.callback(
     Output(component_id='success-pie-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value')]               
)
def update_graph(site_dropdown):
    if (site_dropdown == 'ALL'):
        df  = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(df, names = 'Launch Site',hole=0.5,title = 'Total Success Launches By All Sites')
    else:
        df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(df, names = 'class',hole=.3,title = 'Total Success Launches for site '+site_dropdown)
    return fig

@app.callback(
     Output(component_id='success-vs-payload-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value'),Input(component_id="payload_slider", component_property="value")]               
)
def update_scattergraph(site_dropdown,payload_slider):
    if site_dropdown == 'ALL':
        low, high = payload_slider
        df  = spacex_df
        #mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[(df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)], x="Payload Mass (kg)", y="class", 
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    else:
        low, high = payload_slider
        df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        #mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[(df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)], x="Payload Mass (kg)", y="class", 
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    return fig


if __name__ == '__main__':
    app.run_server()